# EDA Dataset

## Explore

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns   

In [2]:
list1 =np.array(range(10))
list1

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
df_ebay = pd.read_csv('../data/raw/ebayShort.csv', index_col=0)

As the data is too big, we first working in the shortcut of the total data with 1 million rows.

In [4]:
# Look at the first 5 row

df_ebay.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5


In [5]:
#Look at 5 random row
df_ebay.sample(5)

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
242618,C2C,156656,1.0,2019-03-09 09:38:00.000-05:00,1,10.85,2,5,24078,44870,11,14.99,1,2019-03-08 21:17:10.000-05:00,2019-03-12,80,1,PACKAGE_THICK_ENVELOPE,242619
132174,B2C,768932,3.0,2018-01-08 20:04:00.000-08:00,0,2.00,3,5,95121,34638,11,22.00,1,2018-01-03 17:18:05.000-08:00,2018-01-11,11,1,PACKAGE_THICK_ENVELOPE,132175
947870,B2C,37306,1.0,2019-07-25 16:15:00.000-04:00,1,0.00,2,5,11231,76548,9,13.99,1,2019-07-25 12:27:47.000-04:00,2019-07-29,8,1,PACKAGE_THICK_ENVELOPE,947871
744343,B2C,127922,1.0,2018-06-18 16:28:00.000-04:00,1,0.00,2,5,44094,39426,13,44.95,1,2018-06-15 16:52:37.000-04:00,2018-06-20,30,1,PACKAGE_THICK_ENVELOPE,744344
731611,C2C,162387,1.0,2018-02-22 12:30:00.000-05:00,0,0.00,3,5,32080,24501,3,21.00,1,2018-02-21 13:24:43.000-05:00,2018-02-26,3,1,PACKAGE_THICK_ENVELOPE,731612


In [6]:
#check the shape of the dataset
df_ebay.shape

(1000000, 19)

In [7]:
print(f'There are {df_ebay.shape[0]} columns and {df_ebay.shape[1]} rows')

There are 1000000 columns and 19 rows


In [8]:
# Get a quick overview of dataset variables
df_ebay.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   b2c_c2c                    1000000 non-null  object 
 1   seller_id                  1000000 non-null  int64  
 2   declared_handling_days     952820 non-null   float64
 3   acceptance_scan_timestamp  1000000 non-null  object 
 4   shipment_method_id         1000000 non-null  int64  
 5   shipping_fee               1000000 non-null  float64
 6   carrier_min_estimate       1000000 non-null  int64  
 7   carrier_max_estimate       1000000 non-null  int64  
 8   item_zip                   1000000 non-null  object 
 9   buyer_zip                  1000000 non-null  object 
 10  category_id                1000000 non-null  int64  
 11  item_price                 1000000 non-null  float64
 12  quantity                   1000000 non-null  int64  
 13  payment_datet

In [9]:
# top 5 rows showing only 'object' columns
df_ebay.select_dtypes('object').head()

,b2c_c2c,acceptance_scan_timestamp,item_zip,buyer_zip,payment_datetime,delivery_date,package_size
0,B2C,2019-03-26 15:11:00.000-07:00,97219,49040,2019-03-24 03:56:49.000-07:00,2019-03-29,LETTER
1,C2C,2018-06-02 12:53:00.000-07:00,11415-3528,62521,2018-06-01 13:43:54.000-07:00,2018-06-05,PACKAGE_THICK_ENVELOPE
2,B2C,2019-01-07 16:22:00.000-05:00,27292,53010,2019-01-06 00:02:00.000-05:00,2019-01-10,PACKAGE_THICK_ENVELOPE
3,B2C,2018-12-17 16:56:00.000-08:00,90703,80022,2018-12-16 10:28:28.000-08:00,2018-12-21,PACKAGE_THICK_ENVELOPE
4,B2C,2018-07-27 16:48:00.000-07:00,90703,55070,2018-07-26 18:20:02.000-07:00,2018-07-30,PACKAGE_THICK_ENVELOPE


## Data Cleaning

First let do some common checking in the dataset.
 -  Null value
 -  Duplicate row

In [10]:
#Checking null
df_ebay.isna().sum()

b2c_c2c                          0
seller_id                        0
declared_handling_days       47180
acceptance_scan_timestamp        0
shipment_method_id               0
shipping_fee                     0
carrier_min_estimate             0
carrier_max_estimate             0
item_zip                         0
buyer_zip                        0
category_id                      0
item_price                       0
quantity                         0
payment_datetime                 0
delivery_date                    0
weight                           0
weight_units                     0
package_size                     0
record_number                    0
dtype: int64

There is 47180 missing value in total 1 million rows. We could delete it but I decide to have deepper look at it later


In [11]:
# checking duplicate rows
df_ebay.duplicated().any()

False

There are no duplicate row in the database

# Exploring the data

In [12]:
#recall shape column
df_ebay.shape[1]

19

Let check out the distributions, we are going to loop over the columns and print some subplot.